In [1]:
import threading

In [2]:
def task():
    print("hi from task, thread:", threading.get_native_id())

t = threading.Thread(target=task)
t.start()
print("hi from main, thread:", threading.get_native_id()) # thread ID

hi from task, thread:hi from main, thread: 1654
 1667


In [3]:
total = 0
def task2(count):
    global total
    for i in range(count):
        total += 1

t = threading.Thread(target=task2, args=[1_000])
t.start()
t.join() # main thread will block, until t is done
total

1000

In [4]:
total = 0
def task3(count):
    global total
    for i in range(count):
        total += 1

t1 = threading.Thread(target=task3, args=[1_000_000])
t2 = threading.Thread(target=task3, args=[1_000_000])
t1.start()
t2.start()
t1.join() # main thread will block, until t is done
t2.join()
total

1263041

In [5]:
import dis
dis.dis("total += 1")

  0           RESUME                   0

  1           LOAD_NAME                0 (total)
              LOAD_CONST               0 (1)
              BINARY_OP               13 (+=)
              STORE_NAME               0 (total)
              RETURN_CONST             1 (None)


# Locks

In [14]:
%%time
# without lock: 237 ms
# with lock (fine grained): 522 ms
# with lock (coarse grained): 164 ms
lock = threading.Lock() # protects total
total = 0

def task4(count):
    global total
    lock.acquire()
    for i in range(count):
        total += 1
    lock.release()

t1 = threading.Thread(target=task4, args=[1_000_000])
t2 = threading.Thread(target=task4, args=[1_000_000])
t1.start()
t2.start()
t1.join() # main thread will block, until t is done
t2.join()
total

CPU times: user 170 ms, sys: 0 ns, total: 170 ms
Wall time: 169 ms


2000000

In [20]:
bank_accounts = {"x": 25, "y": 100, "z": 200} # in dollars
lock = threading.Lock() # protects bank_accounts

def transfer(src, dst, amount):
    with lock:   # does lock.acquire() for me
        success = False
        if bank_accounts[src] >= amount:
            bank_accounts[src] -= amount
            bank_accounts[dst] += amount
            success = True
        print("transferred" if success else "denied")
    # does lock.release() automatically, even if there is an error

In [21]:
# make sure you always release resoures (files, memory, lock, ...), even if there's an error!

In [22]:
transfer("x", "y", 20)
bank_accounts

transferred


{'x': 5, 'y': 120, 'z': 200}

In [23]:
transfer("x", "y", 20)
bank_accounts

denied


{'x': 5, 'y': 120, 'z': 200}

In [24]:
transfer("w", "y", 20)  # no w in the dict!
bank_accounts

KeyError: 'w'

In [25]:
transfer("x", "y", 1)
bank_accounts

transferred


{'x': 4, 'y': 121, 'z': 200}